In [37]:
import sys
import os
from pathlib import Path
import random
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from contextlib import asynccontextmanager
from pydantic import BaseModel
import pandas as pd
import joblib
from utils import get_model, ModelEnsemble

In [2]:
class ModelEnsemble(BaseEstimator, TransformerMixin):
    """ Class to estimate the premium value for a given client.
    It first predict the ocurrence of a insurance claims, then it proceed to calculate the premium.
    This class is not trainable, it receives models that have been already trained.
    """
    def __init__(self, model1, model2, prime_avg=86.76, n0=42294, n1=5751):
        self.model1 = model1
        self.model2 = model2
        self.prime_avg = prime_avg
        self.n0 = n0
        self.n1 = n1
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        """Method to calculate the premium value for a given set of parameters X.
        PARAMETERS
        ----------
        X: DataFrame
           Dataframe containing the features
        OUTPUT
        ------
        premium: float
               premium value predicted
        proba: list
               probabilities of the possible outcomes               
        """
        model1_output = self.model1.predict(X)
        proba = self.model1.predict_proba(X)[0]
        proba = [round(proba[0], 2), round(proba[1], 2)]
        if model1_output == 0:
            prime = self.prime_avg
        else:
            X["frequence_claims"] = model1_output
            prime = self.prime_avg + self.n1 * np.expm1(self.model2.predict(X)[0]) / self.n0
        return round(prime, 2), proba

In [50]:
path = Path(sys.path[0]).parent
sys.path.insert(1, str(path))
os.chdir(str(path) + '/src')
model = joblib.load('ensemble_model.joblib')

In [68]:
df = pd.DataFrame.from_dict(
    {   'Type':["D"],
        'Occupation':["Housewife"],
        'Age':[59],
        'Group1':[12],
        'Bonus':[-40],
        'Poldur':[1],
        'Value':[48940.0],
        'Adind':[0],
        'Density':[25],
        'Exppdays':[1]
    }
)

prediction, probabability = model.transform(df)
p0, p1 = probabability[0], probabability[1]
output = {"prediction":prediction, "probability":probabability}
print(output)

{'prediction': 86.76, 'probability': [0.85, 0.15]}
